In [1]:
import pandas as pd
from pprint import pprint
import arcgis
from arcgis.gis import GIS
from IPython.display import display

In [2]:
gis = GIS("https://ral.maps.arcgis.com", client_id='9VUcLzqiGGv4M1HS') # Use your own client ID
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ral.maps.arcgis.com/sharing/rest/oauth2/authorize?client_id=9VUcLzqiGGv4M1HS&response_type=code&expiration=-1&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob
Enter code obtained on signing in using SAML: ········
Successfully logged in as: Ryan.Cooper@raleighnc.gov_ral


In [3]:
items = gis.content.search("065e43a8e5884ca1aee06815009ebb84")
data = items[0]

data_layer = data.layers[0]
data_layer_query = data_layer.query(where = '1=1', out_fields = "*", returnGeomtry=False)

df = data_layer_query.df
df.head()

,GEOID10,Join_Count,OBJECTID,POP,Shape__Area,Shape__Length,acres_score,acres_wgt,dist_closest,exp_score,exp_wgt,parks_score,parks_wgt,total_score,SHAPE
0,371830506002010,1,1,33,1.780579e+05,1698.261671,None,104.558587,0.052902,None,42.125,None,17.000,None,"{'rings': [[[2111677.92141655, 739808.84833014..."
1,371830503001003,1,2,194,8.637156e+05,5549.777347,None,114.165090,0.158518,None,38.500,None,12.875,None,"{'rings': [[[2103366.95772356, 744856.98423123..."
2,371830503002010,1,3,13,1.378826e+05,1556.186767,None,185.740984,0.478922,None,39.500,None,14.375,None,"{'rings': [[[2103598.20594873, 740828.71210406..."
3,371830536102038,1,4,2,5.161401e+05,3195.843265,None,9.975569,0.425972,None,5.000,None,1.000,None,"{'rings': [[[2058933.37106597, 783057.45212148..."
4,371830537251011,1,5,481,3.239082e+06,13314.710279,None,54.958079,1.410869,None,4.500,None,1.375,None,"{'rings': [[[2084031.06988622, 773205.79039439..."


In [4]:
def quantileBreakValues(series, quantiles):
    """Define quantile breaks for a pandas series

    Args:
        series (pandas Series): A pandas Series containing numeric values 
                                for which to calculate quantile break values
        quantiles (int): Number of quantiles the Series should be split into
    
    Returns:
        qValuesList: A list of quantile break values for a pandas Series
    
    """
    qValuesList = []
    qInterval = 1/quantiles
    qBreak = 0
    while len(qValuesList) < (quantiles - 1):
        qBreak += qInterval
        qValue = series.quantile(qBreak)
        qValuesList.append(qValue)
    return qValuesList

In [5]:
factor_series = ["acres_wgt", "dist_closest", "exp_wgt", "parks_wgt"]
breaks_dict = {}
[breaks_dict.update({f:quantileBreakValues(df[f], 5)}) for f in factor_series]

pprint(breaks_dict)

{'acres_wgt': [34.8809749162503,
               71.17289503124958,
               94.99770913099995,
               121.22740080825002],
 'dist_closest': [0.3341132297535768,
                  0.6558402065891132,
                  1.057255234717934,
                  1.601113881450642],
 'exp_wgt': [2.75, 8.0, 14.200000000000045, 26.875],
 'parks_wgt': [0.75, 1.625, 3.375, 8.75]}
